In [1]:
import sys, os, math
from collections import Counter
import numpy as np
from scipy.special import beta
from scipy.special import gamma, gammaln, digamma

In [2]:
# represent text as a set of features
def featurizer(data):
    # movie data is already tokenized
    tokens = data.split()
    counter=Counter()
    for t in tokens:
        counter[t]+=1
    return counter


In [3]:
def getData(directory):
    # observation parameters (minimum count for a word to be a feature, max number of total features)
    maxVocab=10000
    minCount=3

    docs = {}
    labels = {}
    totalCounts=Counter()

    featureHash={}
    featureNames=[]

    # read training data and get feature counts and labels for all documents
    for label in ['pos', 'neg']:
        toppath = os.path.join(directory, label)
        for filename in os.listdir(toppath):
            #print filename
            path = os.path.join(toppath, filename)
            data = open(path).read().lower()
            counter=featurizer(data)
            totalCounts+=counter
            docs[filename] = counter
            labels[filename]=label

    # set the feature featureHash
    featureCount=0
    for (word, count) in totalCounts.most_common(maxVocab):
        if count >= minCount:
            featureHash[word]=featureCount
            featureNames.append(word)
            featureCount+=1

    # represent documents as only those words in the common vocabulary (everything else is thrown away)
    numericDocs={}
    for filename in docs:
        numericFeats=Counter()
        for w in docs[filename]:
            if w in featureHash:
                numericFeats[featureHash[w]]=docs[filename][w]

        numericDocs[filename]=numericFeats

    train={}
    dev={}

    # split the data into 90% training, 10% development
    i=0
    for filename in numericDocs:
        if i % 10 == 9:
            dev[filename]=numericDocs[filename]
        else:
            train[filename]=numericDocs[filename]
        i+=1

    return (train, dev, featureNames, labels)


In [4]:
# multinomial naive Bayes models the feature *counts* as multinomials (so that how often a feature shows up with
# data is important)
def multinomialNB(train, dev, featureNames, labels):
    F=len(featureNames)
    values={}
    values["pos"]=0
    values["neg"]=1

    # smooth priors [p(y)] and word likelihoods [p(x|y)]
    priors=np.ones(2, dtype=float)*1.0
    word_likelihoods=np.ones((2, F))*1.0

    # estimate parameters
    for filename in train:
        label=values[labels[filename]]
        for word in train[filename]:
            word_likelihoods[label][word]+=train[filename][word]

        priors[label]+=1.

    priors/=np.sum(priors)
    
    orig=np.ones((2, F))
    orig[0]=list(word_likelihoods[0])
    orig[1]=list(word_likelihoods[1])
    

    word_likelihoods[0]/=np.sum(word_likelihoods[0])
    word_likelihoods[1]/=np.sum(word_likelihoods[1])


    # make predictions
    incorrect=0.

    # use log to avoid numerical underflow (when multiplying many probabilities together)
    for filename in dev:
        class0=math.log(priors[0])
        class1=math.log(priors[1])

        for w in dev[filename]:
            class0+=dev[filename][w]*math.log(word_likelihoods[0][w])
            class1+=dev[filename][w]*math.log(word_likelihoods[1][w])

        trueLabel=values[labels[filename]]

        # prediction is whichever class has the highest probability
        prediction=0
        if class1 > class0:
            prediction=1
        if prediction != trueLabel:
            incorrect+=1


    print "Accuracy: %.3f" % (1-incorrect/len(dev))

    return orig

In [5]:
# bernoulli Naive Bayes models each feature as an independent bernoulli random variable (a binary value of present or absent)

def bernoulliNB(train, dev, featureNames, labels):
    F=len(featureNames)
    values={}
    values["pos"]=0
    values["neg"]=1

    # smooth priors [p(y)] and word likelihoods [p(x|y)]
    priors=np.ones(2, dtype=float)*1.0
    word_likelihoods=np.ones((2, F))*1.0

    # estimate parameters (count number of docs containing each term in vocabulary)
    for filename in train:
        label=values[labels[filename]]
        for word in train[filename]:
            word_likelihoods[label][word]+=1

        priors[label]+=1.

    # normalize each probability distribution
    priors/=np.sum(priors)
    word_likelihoods[0]/=len(train)
    word_likelihoods[1]/=len(train)


    # make predictions
    incorrect=0.

    # use log to avoid numerical underflow (when multiplying many probabilities together)
    for filename in dev:
        class0=math.log(priors[0])
        class1=math.log(priors[1])

        for i in range(F):
            if i in dev[filename]:
                class0+=math.log(word_likelihoods[0][i])
                class1+=math.log(word_likelihoods[1][i])
            else:
                class0+=math.log(1-word_likelihoods[0][i])
                class1+=math.log(1-word_likelihoods[1][i])


        trueLabel=values[labels[filename]]

        # prediction is whichever class has the highest probability
        prediction=0
        if class1 > class0:
            prediction=1
        if prediction != trueLabel:
            incorrect+=1


    print "Accuracy: %.3f" % (1-incorrect/len(dev))

In [15]:
def printFreqs(counts, symmetricAlpha, display):

    F=len(counts[0])

    freqs=np.zeros((2, F))
    freqs[0]=np.array(counts[0])+symmetricAlpha
    freqs[1]=np.array(counts[1])+symmetricAlpha
    
    freqs[0]/=np.sum(freqs[0])
    freqs[1]/=np.sum(freqs[1])
  
    zipped1=zip(freqs[0], featureNames)                     # zip two lists together to iterate through them simultaneously
    zipped1.sort(key = lambda t: t[0], reverse=True)     # sort the two lists by the values in the first (the coefficients)

    zipped2=zip(freqs[1], featureNames)                     # zip two lists together to iterate through them simultaneously
    zipped2.sort(key = lambda t: t[0], reverse=True)     # sort the two lists by the values in the first (the coefficients)


    print "\n## ABSOLUTE FREQUENCIES ##"
    print "%10s\t\t%10s" % ("\nCLASS 1:", "CLASS 2:")
    for i in range(display):
        (weight, word) = zipped1[i]
        (weight2, word2) = zipped2[i]
        
        print "%.5f\t%10s\t%.5f\t%10s" % (weight, word, weight2, word2)



In [16]:
def printFrequencyRatio(counts, symmetricAlpha, display):
    
    F=len(counts[0])
    freqs=np.zeros((2, F))
    freqs[0]=np.array(counts[0])+symmetricAlpha
    freqs[1]=np.array(counts[1])+symmetricAlpha
    
    freqs[0]/=np.sum(freqs[0])
    freqs[1]/=np.sum(freqs[1])
  
    diff=np.zeros(F)
    for i in range(len(diff)):
        diff[i]=math.log(freqs[0][i]/freqs[1][i])
    
    zipped1=zip(diff, featureNames)                     # zip two lists together to iterate through them simultaneously
    zipped1.sort(key = lambda t: t[0], reverse=True)     # sort the two lists by the values in the first (the coefficients)


    print "\n## FREQUENCY RATIO ##"
    print "%10s\t\t%10s" % ("\nCLASS 1:", "CLASS 2:")
    for i in range(display):
        (weight, word) = zipped1[i]
        (weight2, word2) = zipped1[len(zipped1)-i-1]
        
        print "%.5f\t%10s\t%.5f\t%10s" % (weight, word, weight2, word2)




In [17]:
# smoothed log-odds ratio, from:
# Monroe et al. (2009), "Fightin’ Words: Lexical Feature Selection and Evaluation for Identifying the Content of Political Conflict"
# http://languagelog.ldc.upenn.edu/myl/Monroe.pdf
def printLogOddsRatio(counts, symmetricAlpha, display):
    
    F=len(counts[0])

    freqs=np.zeros((2, F))
    freqs[0]=np.array(counts[0])
    freqs[1]=np.array(counts[1])
    
    freqs[0]/=np.sum(freqs[0])
    freqs[1]/=np.sum(freqs[1])
    
    diffs=np.zeros(F)
    sum0=np.sum(counts[0])
    sum1=np.sum(counts[1])
    
    for i in range(F):
        diffs[i]=math.log((counts[0][i] + symmetricAlpha)/(sum0 + F*symmetricAlpha - counts[0][i] - symmetricAlpha)) - math.log((counts[1][i] + symmetricAlpha)/(sum1 + F - counts[1][i] - symmetricAlpha))

    zipped=zip(diffs, featureNames)                     # zip two lists together to iterate through them simultaneously
    zipped.sort(key = lambda t: t[0], reverse=True)     # sort the two lists by the values in the first (the coefficients)

    print "\n## LOG ODDS ##"
    print "%10s\t\t%10s" % ("\nCLASS 1:", "CLASS 2:")
    for i in range(display):
        (weight, word) = zipped[i]
        (weight2, word2) = zipped[len(zipped)-i-1]
        
        print "%.3f\t%10s\t%.3f\t%10s" % (weight, word, weight2, word2)




In [18]:
# smoothed log-odds ratio accounting for variance, from:
# Monroe et al. (2009), "Fightin’ Words: Lexical Feature Selection and Evaluation for Identifying the Content of Political Conflict"
# http://languagelog.ldc.upenn.edu/myl/Monroe.pdf
def printLogOddsRatioWVariance(counts, symmetricAlpha, display):
    
    F=len(counts[0])

    freqs=np.zeros((2, F))
    freqs[0]=np.array(counts[0])
    freqs[1]=np.array(counts[1])
    
    freqs[0]/=np.sum(freqs[0])
    freqs[1]/=np.sum(freqs[1])
    
    diffs=np.zeros(F)
    sum0=np.sum(counts[0])
    sum1=np.sum(counts[1])
    
    for i in range(F):
        diffs[i]=math.log((counts[0][i] + symmetricAlpha)/(sum0 + F*symmetricAlpha - counts[0][i] - symmetricAlpha)) - math.log((counts[1][i] + symmetricAlpha)/(sum1 + F - counts[1][i] - symmetricAlpha))
        diffs[i]/=math.sqrt( (1./(counts[0][i] + symmetricAlpha)) + (1./(counts[1][i] + symmetricAlpha))  )

    zipped=zip(diffs, featureNames)                     # zip two lists together to iterate through them simultaneously
    zipped.sort(key = lambda t: t[0], reverse=True)     # sort the two lists by the values in the first (the coefficients)

    print "\n## LOG ODDS WITH VARIANCE ##"
    print "%10s\t\t%10s" % ("\nCLASS 1:", "CLASS 2:")
    for i in range(display):
        (weight, word) = zipped[i]
        (weight2, word2) = zipped[len(zipped)-i-1]
        
        print "%.3f\t%10s\t%.3f\t%10s" % (weight, word, weight2, word2)



In [10]:
if __name__ == '__main__':

    # path to input directory containing training data
    directory="../data/movie_reviews/txt_sentoken"

In [11]:
    # train and dev are both maps from filename -> dict of feature ids/values
    # featurenNames = array of feature names indexed by feature id
    (train, dev, featureNames, labels) = getData(directory)

In [12]:
    bernoulliNB(train, dev, featureNames, labels)

Accuracy: 0.800


In [13]:
    counts=multinomialNB(train, dev, featureNames, labels)

Accuracy: 0.815


In [19]:
    # print three different views of the learned model
    printFreqs(counts, 1, 10)


## ABSOLUTE FREQUENCIES ##
 
CLASS 1:		  CLASS 2:
0.05581	         ,	0.05151	         ,
0.05425	       the	0.05070	       the
0.04438	         .	0.04708	         .
0.02626	         a	0.02600	         a
0.02601	       and	0.02267	       and
0.02439	        of	0.02258	        of
0.02165	        to	0.02235	        to
0.01851	        is	0.01626	        is
0.01526	        in	0.01457	        in
0.01137	         "	0.01338	         "


In [20]:
    printFrequencyRatio(counts, 1, 10)


## FREQUENCY RATIO ##
 
CLASS 1:		  CLASS 2:
3.15373	     mulan	-3.51830	     &nbsp
3.03989	     shrek	-3.17738	    seagal
2.94316	     flynt	-2.85795	   brenner
2.90332	   gattaca	-2.75616	       8mm
2.74509	    ordell	-2.68206	    sphere
2.62373	     leila	-2.55945	      1900
2.55704	  truman's	-2.51500	       bye
2.55704	 sweetback	-2.51500	   pokemon
2.52195	     taran	-2.49201	schumacher
2.48558	     guido	-2.48423	jawbreaker


In [21]:
    printLogOddsRatio(counts, 1, 10)


## LOG ODDS ##
 
CLASS 1:		  CLASS 2:
3.154	     mulan	-3.518	     &nbsp
3.040	     shrek	-3.177	    seagal
2.943	     flynt	-2.858	   brenner
2.903	   gattaca	-2.756	       8mm
2.745	    ordell	-2.682	    sphere
2.624	     leila	-2.559	      1900
2.557	  truman's	-2.515	       bye
2.557	 sweetback	-2.515	   pokemon
2.522	     taran	-2.492	schumacher
2.486	     guido	-2.484	jawbreaker


In [22]:
    printLogOddsRatioWVariance(counts, 1, 10)


## LOG ODDS WITH VARIANCE ##
 
CLASS 1:		  CLASS 2:
12.477	       and	-18.038	       bad
11.187	         ,	-12.050	         ?
10.596	       his	-11.530	     movie
10.108	      life	-10.799	         i
9.884	        is	-10.605	         *
9.325	       the	-10.474	         !
9.192	        as	-10.452	         "
8.764	         =	-10.446	     worst
8.147	     great	-9.481	    stupid
7.965	       war	-9.303	    boring
